### 1228单天Volume检查

In [ ]:
import pandas as pd
import numpy as np
import os
import re

class Dataconfirm():
    
    def __init__(self, newfolder, oldfolder):
        self.newfolder = newfolder
        self.oldfolder = oldfolder
        
    def checkdata(self):  #每次调取这个函数是一年的
        
        new_path = self.newfolder  #包含年份的一整条绝对路径
        old_path = self.oldfolder

        # year = new_path[ new_path.rindex( '/' ) + 1 : len( new_path ) ] #截取年份
        
        new_timelist = os.listdir(new_path)   #仅遍历获取该日期下所有的时间文件夹名字的集合
        old_timelist = os.listdir(old_path)
        
        AllNewPath = []  #筛选过的完整路径集合
        AllOldPath = []

        for time_i in new_timelist:
            if os.path.exists(old_path + '/' + time_i) == True:
                AllNewPath.append(new_path + '/' + time_i) #保证两个list顺序一致
                AllOldPath.append(old_path + '/' + time_i)
            else:
                new_timelist.remove(time_i) #new_timelist里最后只包含两边都有文件的时间文件名集合,old_timelist也许仍有new里没有的时间
        
        
        os.system("pwd") #确认是从yuqin work出发
        os.system("mkdir -p 2020")
        os.chdir("2020")
        os.system("pwd")
        os.system("mkdir -p 20201228")
        os.chdir("20201228")
        os.system("pwd")
        
        Vol_logname = "1228_Vol.txt"
        Vol_log = open(Vol_logname , 'w')
        
        for n in range(len(AllNewPath)):
            
            getpath_new = AllNewPath[n]
            getpath_old = AllOldPath[n]

            print(getpath_new)
            print(getpath_old)
            
            new_5min = pd.read_csv(getpath_new)
            old_5min = pd.read_csv(getpath_old)
        
            new_colname = list(new_5min.columns)
            new_secucode = list(new_5min.iloc[:,0])
            new_puredata = np.array(new_5min.iloc[:,1:], dtype = float)
            old_colname = list(old_5min.columns)
            old_secucode = list(old_5min.iloc[:,0])
            old_puredata = np.array(old_5min.iloc[:,1:], dtype = float)

            new_index = []
            old_index = []
            
            d = getpath_new
            time = d[ d.rindex( '/' ) + 1 : len( d ) ] 
            withoutLastStr = d[ 0 : d.rindex( '/' ) ]
            date= withoutLastStr[ withoutLastStr.rindex( '/' ) + 1 : len( withoutLastStr ) ]
            year = date[0:4]
            
            for i in range(len(new_secucode)):
                temp = np.where(np.asarray(old_secucode) == new_secucode[i])[0]
                if (len(temp)) > 0:

                    BidVolSumL3Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'BidVolSumL3Twa')[0][0] - 1]
                    BidVolSumL5Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'BidVolSumL5Twa')[0][0] - 1]
                    BidVolSumL10Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'BidVolSumL10Twa')[0][0] - 1]
                    
                    AskVolSumL3Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'AskVolSumL3Twa')[0][0] - 1]
                    AskVolSumL5Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'AskVolSumL5Twa')[0][0] - 1]
                    AskVolSumL10Twa = new_puredata[i, np.where(np.asarray(new_colname) == 'AskVolSumL10Twa')[0][0] - 1]
                    
                    BidVolume = old_puredata[temp, np.where(np.asarray(old_colname) == 'BidVolume')[0][0] - 1]
                    AskVolume = old_puredata[temp, np.where(np.asarray(old_colname) == 'AskVolume')[0][0] - 1]                 
                    
                    new_index.append(i)
                    old_index.append(temp[0])

                        
                    if (np.isnan(BidVolume) == True):
                        continue
                    if not ((BidVolume * 0.1 <= BidVolSumL3Twa) & (BidVolSumL3Twa <= BidVolume * 50)):
                        if (np.isnan(BidVolume) == True) or (np.isnan(BidVolSumL3Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+ time + ' '+ str(BidVolSumL3Twa/BidVolume) +' BidVolSumL3Twa ' + str(BidVolSumL3Twa) + ' & old_bidsize ' + str(BidVolume) + '\r\n')

                        continue
                    if not ((BidVolume * 0.1 <= BidVolSumL5Twa) & (BidVolSumL5Twa <= BidVolume * 50)):
                        if (np.isnan(BidVolume) == True) or (np.isnan(BidVolSumL5Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+ time  +' '+ str(BidVolSumL5Twa/BidVolume) + ' BidVolSumL5Twa ' + str(BidVolSumL5Twa) + ' & old_bidsize ' + str(BidVolume) + '\r\n')

                        continue
                    if not ((BidVolume * 0.1 <= BidVolSumL10Twa) & (BidVolSumL10Twa <= BidVolume * 50)):
                        if (np.isnan(BidVolume) == True) or (np.isnan(BidVolSumL10Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+ time  +' '+ str(BidVolSumL10Twa/BidVolume) + ' BidVolSumL10Twa ' + str(BidVolSumL10Twa) + ' & old_bidsize ' + str(BidVolume) + '\r\n')

                        continue
                        

                    if not ((AskVolume * 0.1 <= AskVolSumL3Twa) & (AskVolSumL3Twa <= AskVolume * 50)):
                        if (np.isnan(AskVolume) == True) or (np.isnan(AskVolSumL3Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+ time  + ' '+str(AskVolSumL3Twa/AskVolume) + ' AskVolSumL3Twa ' + str(AskVolSumL3Twa) + ' & old_asksize ' + str(AskVolume) + '\r\n')
 
                        continue
                    if not ((AskVolume * 0.1 <= AskVolSumL5Twa) & (AskVolSumL5Twa <= AskVolume * 50)):
                        if (np.isnan(AskVolume) == True) or (np.isnan(AskVolSumL5Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+time  +' '+ str(AskVolSumL5Twa/AskVolume) + ' AskVolSumL5Twa ' + str(AskVolSumL5Twa) + ' & old_asksize ' + str(AskVolume) + '\r\n')

                        continue
                    if not ((AskVolume * 0.1 <= AskVolSumL10Twa) & (AskVolSumL10Twa <= AskVolume * 50)):
                        if (np.isnan(AskVolume) == True) or (np.isnan(AskVolSumL10Twa) == True):
                            continue
                        Vol_log.write(str(new_secucode[i])+ ' '+ time  + ' '+str(AskVolSumL10Twa/AskVolume) + ' AskVolSumL10Twa ' + str(AskVolSumL10Twa) + ' & old_asksize ' + str(AskVolume) + '\r\n')

                        continue

            os.chdir("../..")
            
        Vol_log.close
        print('log done')
            
            
def main():
    
    for date in range(20201228, 20201229):
        newfolder = "/dat/all/Equity/CN/marketdata/level2summary/5min/data/2020/" + str(date)
        oldfolder= "/dat/all/Equity/CN/5min/data/2020/" + str(date)
    print(newfolder)
    print(oldfolder)
    
    result = Dataconfirm(newfolder, oldfolder)
    result.checkdata()
    
if __name__ == '__main__':
    
    main()